In [1]:
import os 
import re
import requests
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from datetime import date
from datetime import datetime

from metapub.convert import doi2pmid
from metapub import PubMedFetcher

import pandas as pd
import numpy as np
from lxml import etree
from Bio import Entrez

# make an output directory
######## new for cell type
outdir = '../../results/geo/
# outdir = 'results/hichip_db/'
# os.makedirs(outdir, exist_ok=True)

2023-02-24 16:00:17 herman-login3.hpc.lji.org metapub.config[12320] WARNING NCBI_API_KEY was not set.


## Query the GEO database

In [2]:
# use one of the following search queries/filters
# filters = "HiChIP[All Fields] AND \"gse\"[Filter]" # the whole notebook will take around 7 min to run
# filters = "HiChIP[All Fields] AND (\"gse\"[Filter] AND (\"Homo sapiens\"[Organism] OR \"Mus musculus\"[Organism]))
# filters = "HiChIP[All Fields] AND (\"Homo sapiens\"[Organism] AND \"gse\"[Filter])"
# filters = "HiChIP[All Fields] AND (\"Homo sapiens\"[Organism] AND \"published last year\"[Filter]) AND \"gse\"[Filter]" # 2 min
# filters = "HiChIP[All Fields] AND (\"Mus musculus\"[Organism] AND \"gse\"[Filter])"

# set a dummy email 
Entrez.email = "zjiang@lji.org"

# query the GEO database for HiChIP samples derived from human or mouse
# note: gds is an abbreviation for Gene Expression Omnibus (GEO) database
filters = "HiChIP[All Fields] AND (\"gse\"[Filter] AND (\"Homo sapiens\"[Organism] OR \"Mus musculus\"[Organism]))"
geo_query = Entrez.esearch(db="gds", retmax=10000, term=filters)
geo_results = Entrez.read(geo_query)

# get summary of these entries
geo_ids = ','.join(geo_results['IdList'])
geo_summary = Entrez.esummary(db="gds", id=geo_ids, retmode="xml") 
geo_entries = list(Entrez.parse(geo_summary))

## Obtaining a Map Between PMID and DOI

#### Extract PMIDs

In [4]:
pubmed_ids = set()
for entry in geo_entries:
    for value in entry['PubMedIds']:
        pubmed_ids.add(str(int(value)))
pubmed_ids = sorted(pubmed_ids)
pubmed_ids[0:10]

['27643841',
 '28945252',
 '29224777',
 '29641996',
 '29706538',
 '29731168',
 '30046115',
 '30122536',
 '30397335',
 '30580963']

#### Convert from PMID to DOI using the eutils API

The API was used directly since it was faster than using the metapub package.

In [5]:
# Define the base URL for the efetch command
efetch_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'

# define the parameters for the efetch command
params = {
    'db': 'pubmed',
    'id': ','.join(pubmed_ids),
    'retmode': 'xml',
    'rettype': 'docsum'
}

# send the request to the E-utilities API and get the response
ds_response = requests.get(efetch_url, params=params)

# parse the XML response and extract the DOIs
ds_root = etree.fromstring(ds_response.content)
doc_sums = ds_root.findall('DocSum')
pubmed_data = []
for doc_sum in doc_sums:

    pmid = doc_sum.find('Id').text
    dois = [x.text for x in doc_sum.findall('Item[@Name="DOI"]')]
    dois = ','.join(dois)
    pubmed_data.append([pmid, dois])

pubmed_data = pd.DataFrame(pubmed_data, columns=['PMID', 'DOI'])
pubmed_data.set_index('PMID', inplace=True)
pubmed_data = pubmed_data.squeeze()

## Convert GEO entries into a dataframe

In [8]:
geo_data = []
for i, entry in enumerate(geo_entries):

    ##### Cleaning up the title #####
    paper_title = entry['title']
    new_paper_title = ''
    last_word = re.split('\s|[.]', paper_title)[-1]
    last_two_words = re.split('\s|[.]', paper_title)[-2:]
    
    # remove ".[HiChIP]", ". [HiChIP]", or "(HiChIP)" in paper titles using steps below
    if ('[' in last_word and ']' in last_word) or \
            ('(' in last_word and ')' in last_word):
        new_paper_title = re.split('\s|[.]', paper_title)[0:-1]
        new_paper_title = ' '.join(new_paper_title)
        new_paper_title = new_paper_title.strip()

        # remove the last period in paper title
        if new_paper_title.endswith('.'):
            new_paper_title = new_paper_title[:-1]
            
    # remove ".[Hi ChIP]", ". [Hi ChIP]", or "[Bead Array]" in paper titles using steps below
    elif ('[' in last_two_words[0] and ']' in last_two_words[1]) or \
                ('(' in last_two_words[0] and ')' in last_two_words[1]):
        new_paper_title = re.split('\s|[.]', paper_title)[0:-2]
        new_paper_title = ' '.join(new_paper_title)
        new_paper_title = new_paper_title.strip()

        # remove the last period in paper title
        if new_paper_title.endswith('.'):
            new_paper_title = new_paper_title[:-1]
            
    # remove the period
    elif paper_title.endswith('.'):
        new_paper_title = paper_title[:-1] 
        
    else:
        new_paper_title = paper_title

    new_paper_title = new_paper_title.replace("\xa0", " ") # remove no-break space
    entry['title'] = new_paper_title

    ##### add the doi to the entry #####
    curr_dois = []
    for value in entry['PubMedIds']:
        pubmed_id = str(int(value))
        curr_doi = pubmed_data[pubmed_id]
        curr_dois.append(curr_doi)
    entry['DOI'] = ','.join(curr_dois)

    ##### add the adjusted entry to the data #####
    geo_data.append(entry)
    
geo_df = pd.DataFrame(geo_data)

In [9]:
geo_df.head()

,Item,Id,Accession,GDS,title,summary,GPL,GSE,taxon,entryType,...,n_samples,SeriesTitle,PlatformTitle,PlatformTaxa,SamplesTaxa,PubMedIds,Projects,FTPLink,GEO2R,DOI
0,[],200223259,GSE223259,,Chromatin accessibilities in C9HRE ALS/FTD pat...,This SuperSeries is composed of the SubSeries ...,16791,223259,Homo sapiens,GSE,...,24,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE223nn...,yes,
1,[],200223258,GSE223258,,To profile the chromatin structural changes re...,DAXX is an epigenetic regulator and chromatin ...,16791,223258,Homo sapiens,GSE,...,3,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE223nn...,yes,
2,[],200178673,GSE178673,,Chromatin landscape in perinatal cardiomyocytes,In order to understand the transcriptional reg...,21273;24688,178673,Mus musculus; Rattus norvegicus,GSE,...,28,,,,,"[IntegerElement(36653336, attributes={})]",[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE178nn...,yes,10.1038/s41420-023-01322-3
3,[],200215020,GSE215020,,Generate 3D genome database for atrial and ven...,Measurement the contribution of 3D genome stru...,24247,215020,Mus musculus,GSE,...,6,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE215nn...,yes,
4,[],200215065,GSE215065,,Regulation of mouse chamber selective enhancers,This SuperSeries is composed of the SubSeries ...,19057;26526;24247,215065,synthetic construct; Mus musculus,GSE,...,82,,,,,"[IntegerElement(36705030, attributes={})]",[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE215nn...,yes,10.1161/CIRCULATIONAHA.122.061955


## Reformat the Columns for Google Sheet Compatibility

In [10]:
# drop columns
drop_cols = ['Item', 'Id', 'GDS', 'GPL', 'GSE', 'entryType', 'ptechType',
             'valType', 'SSInfo', 'subsetInfo', 'suppFile', 'Relations', 'ExtRelations',
             'n_samples', 'SeriesTitle', 'PlatformTitle', 'PlatformTaxa', 'SamplesTaxa',
             'Projects', 'FTPLink', 'GEO2R']
geo_df.drop(drop_cols, inplace=True, axis=1)

# rename columns
rename_cols = {"Accession":"GEO / Data link",
               "title": "Paper Title",
               "taxon": "Organism",
               "gdsType": "Any other information",
               "PDAT": "Year",
               "Samples": "Other matched data"}
geo_df.rename(columns=rename_cols, inplace=True)

# extract just the year
geo_df["Year"] = geo_df["Year"].str[:4]

# add index for merging
geo_df['index'] = np.arange(len(geo_df))

# fill na with empty values
geo_df.fillna("", inplace=True)

# adding missing columns with empty values
geo_df["Journal"] = ""
geo_df["Authors"] = ""
geo_df["Tissue/Cell Line"] = ""
geo_df["Presenter"] = ""
geo_df["Potential HiChIP"] = ""

## Assigning Labels to Indicate a Potential HiChIP Sample

In [13]:
# Filter out samples with no HiChIP mention and assigned Yes or Maybe to all others 
lst = []
GSM_IDs = []
index = 0
for row in geo_df["Other matched data"]:
    
    temp = []
    state = False
    
    # Check for the term HiChIP in the title
    for ele in row: 
        if ("HiChIP".casefold() in ele["Title"].casefold()) or \
                ("Hi-ChIP".casefold() in ele["Title"].casefold()):
            state = True
    
    # keep samples with HiChIP and mark "Yes"
    if state == True: 
        for ele in row:
            GSM_IDs.append(ele["Accession"])
            if ("HiChIP".casefold() in ele["Title"].casefold()) or \
                    ("Hi-ChIP".casefold() in ele["Title"].casefold()):
                temp.append(ele["Accession"] + ": " + ele["Title"])
        temp_str = "\n".join(temp)
        geo_df.at[index, "Potential HiChIP"] = "Yes"

    # otherwise, keep all GSM samples and mark "Maybe"
    else: 
        for ele in row:
            GSM_IDs.append(ele["Accession"])
            temp.append(ele["Accession"] + ": " + ele["Title"])
        temp_str = "\n".join(temp)
        geo_df.at[index,"Potential HiChIP"]="Maybe"
    lst.append(temp_str)
    index += 1
    
geo_df["Other matched data"] = list(lst)

## Merge Rows with the Same Paper Title

In [14]:
agg_functions = {'Paper Title':'first',
                    'DOI':'max',
                    'Journal':'first',
                    'Authors':'first',
                    'Year':'first',
                    'GEO / Data link': lambda x: '\n'.join(x),
                    'Any other information':lambda x: '\n'.join(x),
                    'Organism':'first', 'Tissue/Cell Line':'first',
                    'Potential HiChIP':'first',
                    'Other matched data':lambda x: '\n'.join(x),
                    'Presenter':'first'}
geo_df_grouped = geo_df.groupby(geo_df['Paper Title']).aggregate(agg_functions)

## Fetch Additional Paper Metadata

Adding journal name, first author and add https to DOIs

In [15]:
### JR: Extremely slow, need to find a better way to do this

# jounrnal_lst = []
# authors_lst = []
# fetch = PubMedFetcher()
# for DOI in geo_df_grouped['DOI']:
#     try:
#         PMID = doi2pmid(DOI)
#         article = fetch.article_by_pmid(PMID)
#         jounrnal_lst.append(article.journal)
#         authors_lst.append(article.authors[0].split()[0] + " et al.")
#     except:
#         jounrnal_lst.append("")
#         authors_lst.append("")
# geo_df_grouped['Journal'] = jounrnal_lst
# geo_df_grouped['Authors'] = authors_lst

In [112]:
# # add https address to DOI (for easy access)
# geo_df_grouped.loc[(geo_df_grouped['DOI'] != ''), 'DOI'] = 'https://doi.org/' + geo_df_grouped.loc[(geo_df_grouped['DOI'] != ''), 'DOI']

## Save the Final Output File

In [114]:
# add a column that notes the added date
geo_df_grouped['Date Added'] = date_str.replace('_', '-')
reorder = ['Paper Title', 'Journal', 'Authors', 'Year', 'DOI',
           'GEO / Data link', 'Any other information',
           'Organism', 'Tissue/Cell Line', 'Potential HiChIP',
           'Other matched data', 'Presenter', 'Date Added']
geo_df_grouped = geo_df_grouped.loc[:, reorder]

# determining the current year, month and day
today = date.today()
date_str = today.strftime("%Y_%m_%d")

# # determining current hour and minute
# now = datetime.now()
# time_str = now.strftime("%H_%M")

# setting the output filename
#output = os.path.join(outdir, "geo.query.{}_{}.xlsx".format(date_str, time_str))
output = os.path.join(outdir, "geo.query.{}.xlsx".format(date_str))
geo_df_grouped.to_excel(output, index=False)